In [ ]:
from pymilvus import AnnSearchRequest
from sentence_transformers import SentenceTransformer
import pandas as pd
from pymilvus import MilvusClient, RRFRanker
from transformers import AutoTokenizer, AutoModelForCausalLM

embedding_model = SentenceTransformer("Snowflake/snowflake-arctic-embed-s")

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: df20e631-17ba-4dee-8d33-341785dfbe54)')' thrown while requesting HEAD https://huggingface.co/api/resolve-cache/models/Snowflake/snowflake-arctic-embed-s/e596f507467533e48a2e17c007f0e1dacc837b33/tokenizer_config.json
Retrying in 1s [Retry 1/5].


In [2]:
client = MilvusClient("milvus_demo.db")

/home/rahul/github_projects/kh_analytics_interview/.venv/lib/python3.12/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [3]:
def retrieve_chunks(query_text: str):
    query_dense_vector = embedding_model.encode(query_text).tolist()

    search_param_1 = {
        "data": [query_dense_vector],
        "anns_field": "dense_vector",
        "param": {"nprobe": 10},
        "limit": 2,
    }
    request_1 = AnnSearchRequest(**search_param_1)

    search_param_2 = {
        "data": [query_text],
        "anns_field": "sparse_vector",
        "param": {"drop_ratio_search": 0.2},
        "limit": 2,
    }
    request_2 = AnnSearchRequest(**search_param_2)

    reqs = [request_1, request_2]

    chunks = client.hybrid_search(
        collection_name="products_collection",
        reqs=reqs,
        limit=10,
        ranker=RRFRanker(),
        output_fields=["text"],
    )
    return chunks

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-135M-Instruct")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-135M-Instruct")

In [8]:
def generate_answers(contexts, prompt):

    messages = [
        {
            "role": "system",
            "content": "/no_think You are expert in answering questions based on provided context. Make sure to answer only using the context. If the context does not contain the answer, respond with 'I don't know.' /end_no_think",
        },
        {"role": "user", "content": f"Context: {contexts}\n\nQuestion: {prompt}"},
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(**model_inputs, max_new_tokens=32768)

    output_ids = generated_ids[0][len(model_inputs.input_ids[0]) :]

    return tokenizer.decode(output_ids, skip_special_tokens=True)

In [ ]:
prompt = "Tell me about Wayona Nylon Braided USB"

chunks = retrieve_chunks(prompt)
contexts = "\n".join([chunk.entity.get("text") for chunk in chunks[0]])
final_answer = generate_answers(contexts, prompt)
print("Final Answer:", final_answer)

Final Answer: Based on the context, Wayona Nylon Braided USB 2.0 is an innovative and high-quality USB cable designed for iPhones, iPad, and other Apple devices. It features a durable nylon braided design with premium aluminum housing and a toughened nylon fiber wound tightly around the cord, providing superior durability and adding a bit to its flexibility. The cable is designed to fully protect the device from damage from excessive current, and it can charge and sync simultaneously at a rapid speed, making it suitable for iPhones, iPad, and other Apple devices. The cable is also designed to be compatible with any charging adaptor, including power banks, power strips, and power banks, ensuring that the cable can be used with any charging adaptor. Overall, Wayona Nylon Braided USB 2.0 is a reliable and high-quality USB cable that meets the needs of iPhones, iPad, and other Apple devices.
